In [1]:
# Import dependencies
import requests
import pandas as pd
import numpy as np

In [2]:
# Base url for the api call
url = "https://clinicaltrials.gov/api/query/study_fields"

In [3]:
# Headers for json call
headers = {'Content-Type': 'application/json', 
           'Accept': 'application/json'}

In [4]:
# Extract data and create DataFrame
def create_df(x):

    # Create a list to hold the dictionaries from the json response
    sourced_data = []

    # Set a counter to increase the ranks with each iteration
    # Table search criteria limited to 'breast cancer' and from 01/01/2018
    counter = 0

    while True:
        # Define the parameters of the url
        params = {'expr': 'breast cancer AND AREA[StartDate]RANGE[01/01/2018,MAX]', 
                'fields': ','.join(x), 
                'min_rnk': 1 + 1000 * counter,  
                'max_rnk': 1000 + 1000 * counter, 
                'fmt': 'json'}

        # Make the call 
        response = requests.get(url, 
                                headers = headers, 
                                params = params)

        

        # Increment the counter
        counter += 1
        
        # Check to see if more data was returned for the current response
        if response.json()['StudyFieldsResponse']['NStudiesReturned'] == 0:
            # If there was no new data returned, break out of the while loop
            break

        # Add the data from the api call to the sourced_data list
        sourced_data += response.json()['StudyFieldsResponse']['StudyFields']
    return sourced_data

In [5]:
# Define function to clean DataFrame removing unnecessary characters for analysis

def clean_data(df, fields):
    # Change datatype of the values, preparing for reg expression
    df[fields] = df[fields].astype(str)

    # Remove any unnecessary characters and turn blank values with NaN (null)
    for i in fields:
        df[i] = df[i].str.replace("^\[.|.\]$|'","")
        df[i] = df[i].replace('',np.nan)
        

In [6]:
# Define function to clean DataFrame removing unnecessary characters for analysis

def clean_data_years(df, fields):
    # Change datatype of the values, preparing for reg expression
    df[fields] = df[fields].astype(str)

    # Remove any unnecessary characters and turn blank values with NaN (null)
    for i in fields:
        df[i] = df[i].str.replace('Years', '')
        df[i] = df[i].str.replace('Year', '')

In [7]:
# List of fields for registration_info table
reg_info_cols = ['OrgStudyId',
               'BriefTitle',
               'StartDate',
               'CompletionDate',
               'OverallStatus',
               'StudyType']

In [8]:
# Call the function the create DataFrame for registration_info table
registration_df = pd.DataFrame(create_df(reg_info_cols))
registration_df.set_index('Rank', inplace=True)

In [9]:
# Clean DataFrame for table A
clean_data(registration_df, reg_info_cols)

c:\Users\gahyo\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == "__main__":


In [10]:
# Renaming the columns for registration_info table and convertion to datetime and removing duplicates
registration_df.columns = ['ID', 'Title', 'Start_Date', 'Completion_Date', 'Status', 'Study_Type']
registration_df.index.names = ['Index']
registration_df['Start_Date'] = pd.to_datetime(registration_df['Start_Date'])
registration_df['Completion_Date'] = pd.to_datetime(registration_df['Completion_Date'])
registration_df = registration_df.drop_duplicates(subset=['ID'], keep='first')
registration_df

,ID,Title,Start_Date,Completion_Date,Status,Study_Type
Index,,,,,,
1,BTX-BCI-016-PRT,Breast Cancer Index (BCI) Registry,2021-04-14,2028-12-01,Recruiting,Observational
2,2018-TJ-BCD,Diagnosis Value of SEMA4C in Breast Cancer,2023-09-01,2024-09-01,Not yet recruiting,Observational
3,Breast cancer,Role of Sorcin and Annexin A3 in Breast Cancer...,2019-01-20,2019-09-30,Unknown status,Observational
4,BC-BOMET,Evaluation of Prognostic Factors: From Breast ...,2020-01-13,2024-11-12,Recruiting,Observational
5,241391,A Study to Identify Breast Cancer (IDBC),2019-01-24,2022-12-31,Unknown status,Observational
...,...,...,...,...,...,...
5025,32900654326,"TPVB, PECSB, ESPB for Postmastectmy Pain",2019-04-10,2021-08-10,Completed,Interventional
5026,ReDA 13176,A Randomized Phase III Trial of Stereotactic A...,2023-04-01,2029-04-01,Not yet recruiting,Interventional
5027,64121317.4.1001.5330,Nivolumab in Prostate Cancer With DNA Repair D...,2018-06-01,2022-03-01,"Active, not recruiting",Interventional


In [11]:
# add new column 'Actual Duration' calculating the difference between start date and completion date 
registration_df['Actual_Duration'] = (registration_df['Completion_Date'] - registration_df['Start_Date'])
registration_df['Actual_Duration'] = registration_df['Actual_Duration'].dt.total_seconds() / (365*24*60*60)

#------------------------------------------------------

# Reorder the columns 
registration_df = registration_df[['ID', 'Title', 'Start_Date', 'Completion_Date', 'Actual_Duration', 'Status', 'Study_Type']]
registration_df


,ID,Title,Start_Date,Completion_Date,Actual_Duration,Status,Study_Type
Index,,,,,,,
1,BTX-BCI-016-PRT,Breast Cancer Index (BCI) Registry,2021-04-14,2028-12-01,7.638356,Recruiting,Observational
2,2018-TJ-BCD,Diagnosis Value of SEMA4C in Breast Cancer,2023-09-01,2024-09-01,1.002740,Not yet recruiting,Observational
3,Breast cancer,Role of Sorcin and Annexin A3 in Breast Cancer...,2019-01-20,2019-09-30,0.693151,Unknown status,Observational
4,BC-BOMET,Evaluation of Prognostic Factors: From Breast ...,2020-01-13,2024-11-12,4.835616,Recruiting,Observational
5,241391,A Study to Identify Breast Cancer (IDBC),2019-01-24,2022-12-31,3.936986,Unknown status,Observational
...,...,...,...,...,...,...,...
5025,32900654326,"TPVB, PECSB, ESPB for Postmastectmy Pain",2019-04-10,2021-08-10,2.336986,Completed,Interventional
5026,ReDA 13176,A Randomized Phase III Trial of Stereotactic A...,2023-04-01,2029-04-01,6.005479,Not yet recruiting,Interventional
5027,64121317.4.1001.5330,Nivolumab in Prostate Cancer With DNA Repair D...,2018-06-01,2022-03-01,3.750685,"Active, not recruiting",Interventional


In [12]:
# List of fields for participant table
participant_cols = ['OrgStudyId',
               'Gender',
               'MinimumAge',
               'HealthyVolunteers']

In [13]:
# Call the function the create DataFrame for participant table
participant_df = pd.DataFrame(create_df(participant_cols))
participant_df.set_index('Rank', inplace=True)

In [14]:
# Clean DataFrame for participant table
clean_data(participant_df, participant_cols)
clean_data_years(participant_df, participant_cols)

c:\Users\gahyo\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == "__main__":


In [15]:
# Minimum Age: Change any months values into 0 years
for i, rows in participant_df.iterrows():
    if 'Months' in participant_df.loc[i].MinimumAge:
        participant_df.loc[i].MinimumAge = 0
    elif 'Month' in participant_df.loc[i].MinimumAge:
        participant_df.loc[i].MinimumAge = 0
    else:
        continue

In [16]:
# Renaming the columns for participant table and getting rid of rows
participant_df.columns = ['ID', 'Gender', 'Minimum_Age', 'Healthy_Volunteers']
participant_df.index.names = ['Index']
participant_df = participant_df.drop_duplicates(subset=['ID'], keep='first')
participant_df

# ** below code can be used if Minimum Age needs to be converted into integer
#  participant_df['Minimum_Age'] = participant_df['Minimum_Age'].astype(float).astype('Int64')


,ID,Gender,Minimum_Age,Healthy_Volunteers
Index,,,,
1,BTX-BCI-016-PRT,Female,18,No
2,2018-TJ-BCD,Female,18,No
3,Breast cancer,Female,20,Accepts Healthy Volunteers
4,BC-BOMET,Female,18,No
5,241391,Female,30,Accepts Healthy Volunteers
...,...,...,...,...
5025,32900654326,Female,18,No
5026,ReDA 13176,All,18,No
5027,64121317.4.1001.5330,Male,18,No


In [17]:
# List of fields for study detail table
study_detail_cols = ['OrgStudyId',
               'IsFDARegulatedDrug',
               'IsFDARegulatedDevice',
               'ResponsiblePartyType']

In [18]:
# Call the function the create DataFrame for study method table
study_details_df = pd.DataFrame(create_df(study_detail_cols))
study_details_df.set_index('Rank', inplace=True)

In [19]:
# Clean DataFrame for study method table
clean_data(study_details_df, study_detail_cols)

c:\Users\gahyo\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == "__main__":


In [20]:
# Renaming the columns for study detail table and removing duplicates
study_details_df.columns = ['ID', 'FDA_Regulated_Drug', 'FDA_Regulated_Device', 'Responsible_Party']
study_details_df.index.names = ['Index']
study_details_df = study_details_df.drop_duplicates(subset=['ID'], keep='first')
study_details_df

,ID,FDA_Regulated_Drug,FDA_Regulated_Device,Responsible_Party
Index,,,,
1,BTX-BCI-016-PRT,No,No,Sponsor
2,2018-TJ-BCD,No,No,Principal Investigator
3,Breast cancer,No,No,Principal Investigator
4,BC-BOMET,No,No,Sponsor
5,241391,No,No,Sponsor
...,...,...,...,...
5025,32900654326,No,No,Principal Investigator
5026,ReDA 13176,No,No,Sponsor-Investigator
5027,64121317.4.1001.5330,Yes,No,Sponsor


In [21]:
# List of fields for study method table
study_method_cols = ['OrgStudyId',
          'StudyType',
          'ArmGroupType',
          'InterventionType',
          'DesignInterventionModel',
          'DesignObservationalModel',
          'TargetDuration',
          'SamplingMethod',
          'Phase']

In [22]:
# Call function to create DataFrame for table D
study_method_df = pd.DataFrame(create_df(study_method_cols))
study_method_df.set_index('Rank',inplace=True)

In [23]:
# Clean DataFrame for study method table
clean_data(study_method_df, study_method_cols)

c:\Users\gahyo\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == "__main__":


In [24]:
# Renaming the columns for table D and removing duplicates and simplifyingstudy_method_df.columns = ['ID', 'Study_Type', 'Arm_Group_Type', 'Intervention_Type', 'Interventional_Study_Model', 'Observational_Study_Model', 'Target_Duration', 'Sampling_Method', 'Phase']
study_method_df.index.names = ['Index']
study_method_df.columns = ['ID', 'Study_Type', 'Arm_Group_Type', 'Intervention_Type', 'Interventional_Study_Model', 'Observational_Study_Model', 'Target_Duration', 'Sampling_Method', 'Phase']
study_method_df = study_method_df.drop_duplicates(subset=['ID'], keep='first')
study_method_df['Arm_Group_Type'] = study_method_df['Arm_Group_Type'].str.split(',').str[0]
study_method_df['Intervention_Type'] = study_method_df['Intervention_Type'].str.split(',').str[0]
study_method_df

,ID,Study_Type,Arm_Group_Type,Intervention_Type,Interventional_Study_Model,Observational_Study_Model,Target_Duration,Sampling_Method,Phase
Index,,,,,,,,,
1,BTX-BCI-016-PRT,Observational,NaN,Diagnostic Test,NaN,Case-Only,5 Years,Non-Probability Sample,NaN
2,2018-TJ-BCD,Observational,NaN,Diagnostic Test,NaN,Case-Control,NaN,Non-Probability Sample,NaN
3,Breast cancer,Observational,NaN,Genetic,NaN,Case-Control,NaN,Probability Sample,NaN
4,BC-BOMET,Observational,NaN,Other,NaN,Case-Control,NaN,Non-Probability Sample,NaN
5,241391,Observational,NaN,Diagnostic Test,NaN,Cohort,1 Year,Non-Probability Sample,NaN
...,...,...,...,...,...,...,...,...,...
5025,32900654326,Interventional,Placebo Comparator,Procedure,Parallel Assignment,NaN,NaN,NaN,Not Applicable
5026,ReDA 13176,Interventional,Active Comparator,Radiation,Parallel Assignment,NaN,NaN,NaN,Phase 3
5027,64121317.4.1001.5330,Interventional,Experimental,Drug,Single Group Assignment,NaN,NaN,NaN,Phase 2


In [25]:
# List of fields for study detail table and export dataset into DataFrame
text_analysis_cols = ['OrgStudyId',
          'WhyStopped',            
          'EnrollmentCount',
          'PrimaryOutcomeMeasure',]
text_analysis_df = pd.DataFrame(create_df(text_analysis_cols))
text_analysis_df.set_index('Rank', inplace=True)


In [26]:
clean_data(text_analysis_df, text_analysis_cols)

c:\Users\gahyo\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == "__main__":


In [27]:
# Renaming the columns for table E and removing duplicates
text_analysis_df.columns = ['ID', 'Why_Stopped','Enrollment_Count', 'Primary_Outcome_Measure']
text_analysis_df.index.names = ['Index']
text_analysis_df = text_analysis_df.drop_duplicates(subset=['ID'], keep='first')
text_analysis_df

,ID,Why_Stopped,Enrollment_Count,Primary_Outcome_Measure
Index,,,,
1,BTX-BCI-016-PRT,NaN,3000,To determine BCI test performance by evaluatin...
2,2018-TJ-BCD,NaN,2300,Diagnostic potential of SEMA4C as a biomarker ...
3,Breast cancer,NaN,80,Role of SORCIN in patients with breast cancer
4,BC-BOMET,NaN,30,SENP1 expression
5,241391,NaN,600,Performance of the Syantra DX Breast Cancer te...
...,...,...,...,...
5025,32900654326,NaN,80,The primary outcome of the study will be durat...
5026,ReDA 13176,NaN,180,Overall Survival
5027,64121317.4.1001.5330,NaN,38,PSA response rate


In [28]:
# Export DataFrames as csv files 
study_method_df.to_csv('Tables/study_method_df.csv',index=False)
text_analysis_df.to_csv('Tables/free_text_df.csv', index=False)
registration_df.to_csv('Tables/registration_df.csv', index=False)
participant_df.to_csv('Tables/participant_df.csv', index=False)
study_details_df.to_csv('Tables/study_details_df.csv', index=False)